![Parked car](car.jpg)

Insurance companies invest a lot of [time and money](https://www.accenture.com/_acnmedia/pdf-84/accenture-machine-leaning-insurance.pdf) into optimizing their pricing and accurately estimating the likelihood that customers will make a claim. In many countries, insurance is a legal requirement to have car insurance in order to drive a vehicle on public roads, so the market is very large!

Knowing all of this, On the Road car insurance has requested your services in building a model to predict whether a customer will make a claim on their insurance during the policy period. As they have very little expertise and infrastructure for deploying and monitoring machine learning models, they've asked you to use simple Logistic Regression, identifying the single feature that results in the best-performing model, as measured by accuracy.

They have supplied you with their customer data as a csv file called `car_insurance.csv`, along with a table (below) detailing the column names and descriptions below.

## The dataset

| Column | Description |
|--------|-------------|
| `id` | Unique client identifier |
| `age` | Client's age: <br> <ul><li>`0`: 16-25</li><li>`1`: 26-39</li><li>`2`: 40-64</li><li>`3`: 65+</li></ul> |
| `gender` | Client's gender: <br> <ul><li>`0`: Female</li><li>`1`: Male</li></ul> |
| `driving_experience` | Years the client has been driving: <br> <ul><li>`0`: 0-9</li><li>`1`: 10-19</li><li>`2`: 20-29</li><li>`3`: 30+</li></ul> |
| `education` | Client's level of education: <br> <ul><li>`0`: No education</li><li>`1`: High school</li><li>`2`: University</li></ul> |
| `income` | Client's income level: <br> <ul><li>`0`: Poverty</li><li>`1`: Working class</li><li>`2`: Middle class</li><li>`3`: Upper class</li></ul> |
| `credit_score` | Client's credit score (between zero and one) |
| `vehicle_ownership` | Client's vehicle ownership status: <br><ul><li>`0`: Does not own their vehilce (paying off finance)</li><li>`1`: Owns their vehicle</li></ul> |
| `vehcile_year` | Year of vehicle registration: <br><ul><li>`0`: Before 2015</li><li>`1`: 2015 or later</li></ul> |
| `married` | Client's marital status: <br><ul><li>`0`: Not married</li><li>`1`: Married</li></ul> |
| `children` | Client's number of children |
| `postal_code` | Client's postal code | 
| `annual_mileage` | Number of miles driven by the client each year |
| `vehicle_type` | Type of car: <br> <ul><li>`0`: Sedan</li><li>`1`: Sports car</li></ul> |
| `speeding_violations` | Total number of speeding violations received by the client | 
| `duis` | Number of times the client has been caught driving under the influence of alcohol |
| `past_accidents` | Total number of previous accidents the client has been involved in |
| `outcome` | Whether the client made a claim on their car insurance (response variable): <br><ul><li>`0`: No claim</li><li>`1`: Made a claim</li></ul> |

In [20]:
# Import required libraries
library(readr)
library(dplyr)
library(glue)
library(yardstick)

## Investigate and clean the data, so that there are no missing values and remove the "id" column.

In [21]:
# Load the data
df.car_insurance <- read.csv('car_insurance.csv')

# Display the first few rows
head(df.car_insurance)

# Summary of data types
str(df.car_insurance)
colnames(df.car_insurance)

# Basic statistical summary 
summary(df.car_insurance)

,id,age,gender,race,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>
1,569520,3,0,1,0,2,3,0.6290273,1,1,0,1,10238,12000,0,0,0,0,0
2,750365,0,1,1,0,0,0,0.3577571,0,0,0,0,10238,16000,0,0,0,0,1
3,199901,0,0,1,0,2,1,0.4931458,1,0,0,0,10238,11000,0,0,0,0,0
4,478866,0,1,1,0,3,1,0.2060129,1,0,0,1,32765,11000,0,0,0,0,0
5,731664,1,1,1,1,0,1,0.3883659,1,0,0,0,32765,12000,0,2,0,1,1
6,877557,2,0,1,2,2,3,0.6191274,1,1,0,1,10238,13000,0,3,0,3,0


'data.frame':	10000 obs. of  19 variables:
 $ id                 : int  569520 750365 199901 478866 731664 877557 930134 461006 68366 445911 ...
 $ age                : int  3 0 0 0 1 2 3 1 2 2 ...
 $ gender             : int  0 1 0 1 1 0 1 0 0 0 ...
 $ race               : int  1 1 1 1 1 1 1 1 1 1 ...
 $ driving_experience : int  0 0 0 0 1 2 3 0 2 0 ...
 $ education          : int  2 0 2 3 0 2 2 3 3 2 ...
 $ income             : int  3 0 1 1 1 3 3 1 1 3 ...
 $ credit_score       : num  0.629 0.358 0.493 0.206 0.388 ...
 $ vehicle_ownership  : num  1 0 1 1 1 1 0 0 0 1 ...
 $ vehicle_year       : int  1 0 0 0 0 1 1 1 0 0 ...
 $ married            : num  0 0 0 0 0 0 1 0 1 0 ...
 $ children           : num  1 0 0 1 0 1 1 1 0 1 ...
 $ postal_code        : int  10238 10238 10238 32765 32765 10238 10238 10238 10238 32765 ...
 $ annual_mileage     : num  12000 16000 11000 11000 12000 13000 13000 14000 13000 11000 ...
 $ vehicle_type       : int  0 0 0 0 0 0 0 0 0 0 ...
 $ speeding_violations:

[1] "id"                  "age"                 "gender"             
 [4] "race"                "driving_experience"  "education"          
 [7] "income"              "credit_score"        "vehicle_ownership"  
[10] "vehicle_year"        "married"             "children"           
[13] "postal_code"         "annual_mileage"      "vehicle_type"       
[16] "speeding_violations" "duis"                "past_accidents"     
[19] "outcome"

       id              age           gender           race       
 Min.   :   101   Min.   :0.00   Min.   :0.000   Min.   :0.0000  
 1st Qu.:249638   1st Qu.:1.00   1st Qu.:0.000   1st Qu.:1.0000  
 Median :501777   Median :1.00   Median :0.000   Median :1.0000  
 Mean   :500522   Mean   :1.49   Mean   :0.499   Mean   :0.9012  
 3rd Qu.:753974   3rd Qu.:2.00   3rd Qu.:1.000   3rd Qu.:1.0000  
 Max.   :999976   Max.   :3.00   Max.   :1.000   Max.   :1.0000  
                                                                 
 driving_experience   education        income     credit_score   
 Min.   :0.000      Min.   :0.00   Min.   :0.0   Min.   :0.0534  
 1st Qu.:0.000      1st Qu.:2.00   1st Qu.:1.0   1st Qu.:0.4172  
 Median :1.000      Median :2.00   Median :2.0   Median :0.5250  
 Mean   :1.069      Mean   :2.01   Mean   :1.9   Mean   :0.5158  
 3rd Qu.:2.000      3rd Qu.:3.00   3rd Qu.:3.0   3rd Qu.:0.6183  
 Max.   :3.000      Max.   :3.00   Max.   :3.0   Max.   :0.9608  
          

In [22]:
# Count the number of missing values (NA) in each column of the dataframe
na_count <- colSums(is.na(df.car_insurance))
na_count

id                 age              gender                race 
                  0                   0                   0                   0 
 driving_experience           education              income        credit_score 
                  0                   0                   0                 982 
  vehicle_ownership        vehicle_year             married            children 
                  0                   0                   0                   0 
        postal_code      annual_mileage        vehicle_type speeding_violations 
                  0                 957                   0                   0 
               duis      past_accidents             outcome 
                  0                   0                   0

In [23]:
# load tidyverse to use drop_na
library(tidyr)

# remove rows with missing values
df.car_insurance <- df.car_insurance %>% drop_na()

# check there are no missing values (NA) in the dataframe
na_count <- colSums(is.na(df.car_insurance))
na_count

id                 age              gender                race 
                  0                   0                   0                   0 
 driving_experience           education              income        credit_score 
                  0                   0                   0                   0 
  vehicle_ownership        vehicle_year             married            children 
                  0                   0                   0                   0 
        postal_code      annual_mileage        vehicle_type speeding_violations 
                  0                   0                   0                   0 
               duis      past_accidents             outcome 
                  0                   0                   0

In [24]:
# remove the "id" column
df.car_insurance_noid <- subset(df.car_insurance, select = -c(id))

# display the first few rows to confirm removed column
head(df.car_insurance_noid)

,age,gender,race,driving_experience,education,income,credit_score,vehicle_ownership,vehicle_year,married,children,postal_code,annual_mileage,vehicle_type,speeding_violations,duis,past_accidents,outcome
,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>
1,3,0,1,0,2,3,0.6290273,1,1,0,1,10238,12000,0,0,0,0,0
2,0,1,1,0,0,0,0.3577571,0,0,0,0,10238,16000,0,0,0,0,1
3,0,0,1,0,2,1,0.4931458,1,0,0,0,10238,11000,0,0,0,0,0
4,0,1,1,0,3,1,0.2060129,1,0,0,1,32765,11000,0,0,0,0,0
5,1,1,1,1,0,1,0.3883659,1,0,0,0,32765,12000,0,2,0,1,1
6,2,0,1,2,2,3,0.6191274,1,1,0,1,10238,13000,0,3,0,3,0


## Find the feature with the best predictive performance for a car insurance claim ("outcome") by creating simple Logistic Regression models (each with a single feature) and assessing their accuracy.

In [25]:
# Create a dataframe to store features
features_df <- data.frame(features = c(names(subset(df.car_insurance, select = -c(id, outcome)))))

# Empty vector to store accuracies
accuracies <- c()

# Loop through features
for (col in features_df$features) {
    # Create a binomial general linear model (GLM)
    model <- glm(glue('outcome ~ {col}'), data = df.car_insurance, family = 'binomial')
    # Get prediction values for the model
    predictions <- round(fitted(model))
    # Calculate accuracy
    accuracy <- length(which(predictions == df.car_insurance$outcome)) / length(df.car_insurance$outcome)
	# Add accuracy to features_df
	features_df[which(features_df$feature == col), "accuracy"] = accuracy
}

# print the features dataframe to see the accuracies for each variable in ascending order
features_df[order(-features_df$accuracy), ]

,features,accuracy
,<chr>,<dbl>
4,driving_experience,0.7781323
1,age,0.7740827
6,income,0.7399681
8,vehicle_ownership,0.7392318
7,credit_score,0.7065898
13,annual_mileage,0.6927230
2,gender,0.6887962
3,race,0.6887962
5,education,0.6887962


## Create a data frame called best_feature_df, containing columns named "best_feature" and "best_accuracy" with the name of the feature with the highest accuracy, and the respective accuracy score.

In [26]:
# Create a dataframe called best_features_df and find the feature with the largest accuracy, i.e. the max value.
best_feature <- features_df$features[which.max(features_df$accuracy)]
best_accuracy <- max(features_df$accuracy)

# Create best_feature_df
best_feature_df <- data.frame(best_feature, best_accuracy)

# Run in a new cell to check your solution
best_feature_df

best_feature,best_accuracy
<chr>,<dbl>
driving_experience,0.7781323
